<h1 align=center><font size = 5>
Week 3 Assignment of Applied Data Science Capstone
</font></h1>


<h1 align=center><font size = 5>Task 2: Add the latitude and longitude coordinates to the dataframe</font></h1>


In [1]:
import pandas as pd

Let's read our csv file we created at the end of the last notebook

In [2]:
df = pd.read_csv('first_task')

In [3]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


we will use the Geocoder Python package : https://geocoder.readthedocs.io/index.html.we  in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [4]:
!pip install geocoder
import geocoder


     |████████████████████████████████| 98 kB 304 kB/s eta 0:00:01


### Use Arcgis to get the coordinates

In [12]:
def get_lalo(postal_code):
    
    la_lo_coords = None
    
    while(la_lo_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        la_lo_coords = g.latlng
    return la_lo_coords
    
get_lalo('M4G')

[43.709020000000066, -79.36348999999996]

Let's retrieve postal code coordinates

In [13]:
postal_codes = df['PostalCode']    
coordinates = [ get_lalo(postal_code) for postal_code in postal_codes.tolist() ]


make new columns for Latitude and Longitude

In [14]:
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']


In [16]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [17]:
df[df.PostalCode == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493


Save new dataset to CSV file

In [18]:
df.to_csv('2nd_task.csv', index=False)


In [19]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892
...,...,...,...,...,...
98,M9N,York,Weston,43.70507,-79.51804
99,M9P,Etobicoke,Westmount,43.69630,-79.52926
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.68681,-79.55728
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.74453,-79.58624


# Task 3: Explore and cluster the neighborhoods in Torondo

#### Import libraries and install folium library 

In [20]:
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.0
  latest version: 4.9.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/moh/anaconda3/envs/moh

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         145 KB

The following packages will be UPDATED:

  ca-certificates                      2020.6.20-hecda079_0 --> 2020.11.8-ha878542_0



ca-certificates-2020 | 145 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [21]:
toronto_coords = get_lalo('')
toronto_coords

[43.648690000000045, -79.38543999999996]

#### work only with boroughs that contain the word 'Toronto'

In [24]:
dfT = df[ df.Borough.str.contains('Toronto') ]

In [26]:
dfT.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.67709,-79.29547
41,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31467
43,M4M,East Toronto,Studio District,43.66213,-79.33497
44,M4N,Central Toronto,Lawrence Park,43.72843,-79.38713
45,M4P,Central Toronto,Davisville North,43.71276,-79.38851
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.71458,-79.40668
47,M4S,Central Toronto,Davisville,43.70340,-79.38659
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.69048,-79.38318
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.68568,-79.40237


Creade the map for dataset

In [30]:
map_toronto = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfT['Latitude'], dfT['Longitude'], dfT['Borough'], dfT['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [31]:
toronto_data = pd.DataFrame(dfT)

In [32]:
toronto_data = toronto_data.reset_index().drop('index', axis=1)

In [34]:
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.67709,-79.29547
1,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31467
3,M4M,East Toronto,Studio District,43.66213,-79.33497
4,M4N,Central Toronto,Lawrence Park,43.72843,-79.38713


In [35]:
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [36]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] 

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] 

print(neighborhood_name, neighborhood_latitude, neighborhood_longitude)


The Beaches 43.67709000000008 -79.29546999999997


Top 100 venues that are in East Toronto within a radius of 500 meters.

In [37]:
# Let's create GET request URL
VERSION = '20180605'
CLIENT_ID = 'MT4MYN0ZDBBWNKBIT2ATB0KPBAVD4IPTGSODCMKDGY44VQRB'
CLIENT_SECRET = 'ML51XFIPC3A5NT1EKIIBKTSHP3DDJLBMIOIDVENJYMDTII5N'
latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,radius, LIMIT)


In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fabd2d5f5f91049c0b62b35'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.68159000450008,
    'lng': -79.28925963866925},
   'sw': {'lat': 43.672589995500076, 'lng': -79.30168036133068}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


<ipython-input-40-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869



### Explore Neighborhoods in Toronto
### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [42]:
toronto_venues = getNearbyVenues(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude'])
# create a new dataframe called toronto_venues and run the above function on each neighborhood

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [43]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.67709,-79.29547,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.67709,-79.29547,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.67709,-79.29547,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.67709,-79.29547,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.68375,-79.35512,Dairy Queen,43.684223,-79.357062,Ice Cream Shop
5,"The Danforth West, Riverdale",43.68375,-79.35512,Centennial College - Story Arts Centre,43.684874,-79.349140,Business Service
6,"The Danforth West, Riverdale",43.68375,-79.35512,Dollarama,43.686197,-79.355989,Discount Store
7,"The Danforth West, Riverdale",43.68375,-79.35512,Pottery & Broadview,43.683712,-79.357101,Intersection
8,"The Danforth West, Riverdale",43.68375,-79.35512,Charles Sauriol Parkette,43.685270,-79.356588,Park
9,"The Danforth West, Riverdale",43.68375,-79.35512,TTC Bus #8 Broadview,43.687101,-79.355078,Bus Line


Let's find out our dataset's size

In [44]:
toronto_venues.shape

(1738, 7)

In [45]:
# check out how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",85,85,85,85,85,85
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",76,76,76,76,76,76
Central Bay Street,76,76,76,76,76,76
Christie,11,11,11,11,11,11
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26


## Analyze Each Neighborhood

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [47]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,...,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529,...,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.010000,...,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.013158,0.000000,0.000000,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.012658,0.000000,0.012658,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.00
7,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00



### Let's put that into a pandas dataframe

### First, let's write a function to sort the venues in descending order.


In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Farmers Market,Bakery,Restaurant,Breakfast Spot,Cheese Shop,Beer Bar,Cocktail Bar,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Gift Shop,Sandwich Place,Nightclub,Lounge,Japanese Restaurant,Breakfast Spot
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Café,Bar,Italian Restaurant,Asian Restaurant,Seafood Restaurant,Steakhouse,Sushi Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Café,Coffee Shop,Bar,French Restaurant,Park,Restaurant,Speakeasy,Lounge,Bakery
4,Central Bay Street,Coffee Shop,Clothing Store,Sushi Restaurant,Middle Eastern Restaurant,Plaza,Café,Restaurant,Bubble Tea Shop,Hotel,Bookstore
5,Christie,Café,Grocery Store,Playground,Candy Store,Italian Restaurant,Baby Store,Athletics & Sports,Coffee Shop,Farm,Electronics Store
6,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café,Sushi Restaurant,Mediterranean Restaurant,Dance Studio,Pub,Men's Store
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gym,Japanese Restaurant,Beer Bar,American Restaurant,Seafood Restaurant
8,Davisville,Dessert Shop,Italian Restaurant,Café,Sandwich Place,Pizza Place,Coffee Shop,Thai Restaurant,Farmers Market,Fast Food Restaurant,Seafood Restaurant
9,Davisville North,Dog Run,Gym / Fitness Center,Hotel,Breakfast Spot,Food & Drink Shop,Department Store,Park,Japanese Restaurant,Event Space,Elementary School


## 4. Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 5 clusters.


In [50]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [59]:

# add clustering labels

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.67709,-79.29547,0.0,Health Food Store,Pub,Trail,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
1,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512,0.0,Coffee Shop,Discount Store,Bus Line,Business Service,Grocery Store,Park,Ice Cream Shop,Intersection,Elementary School,Escape Room
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31467,0.0,Park,Italian Restaurant,Pet Store,Gym,Pizza Place,Pub,Coffee Shop,Restaurant,Movie Theater,Sandwich Place
3,M4M,East Toronto,Studio District,43.66213,-79.33497,0.0,Brewery,Pizza Place,Diner,Italian Restaurant,Gastropub,Bar,Coffee Shop,Arts & Crafts Store,Sushi Restaurant,Bakery
4,M4N,Central Toronto,Lawrence Park,43.72843,-79.38713,1.0,Bus Line,Swim School,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


Finally, let's visualize the resulting clusters


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


### Examine Clusters
#### Cluster 1

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0.0,Health Food Store,Pub,Trail,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
1,East Toronto,0.0,Coffee Shop,Discount Store,Bus Line,Business Service,Grocery Store,Park,Ice Cream Shop,Intersection,Elementary School,Escape Room
2,East Toronto,0.0,Park,Italian Restaurant,Pet Store,Gym,Pizza Place,Pub,Coffee Shop,Restaurant,Movie Theater,Sandwich Place
3,East Toronto,0.0,Brewery,Pizza Place,Diner,Italian Restaurant,Gastropub,Bar,Coffee Shop,Arts & Crafts Store,Sushi Restaurant,Bakery
5,Central Toronto,0.0,Dog Run,Gym / Fitness Center,Hotel,Breakfast Spot,Food & Drink Shop,Department Store,Park,Japanese Restaurant,Event Space,Elementary School
7,Central Toronto,0.0,Dessert Shop,Italian Restaurant,Café,Sandwich Place,Pizza Place,Coffee Shop,Thai Restaurant,Farmers Market,Fast Food Restaurant,Seafood Restaurant
9,Central Toronto,0.0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
10,Downtown Toronto,0.0,Tennis Court,Playground,Park,Bike Trail,Shop & Service,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
11,Downtown Toronto,0.0,Coffee Shop,Café,Restaurant,Pizza Place,Park,Chinese Restaurant,Italian Restaurant,Pub,Market,Bakery
12,Downtown Toronto,0.0,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café,Sushi Restaurant,Mediterranean Restaurant,Dance Studio,Pub,Men's Store


### Cluster 2

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,1.0,Bus Line,Swim School,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Cluster 3

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,2.0,Playground,Gym Pool,Park,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
23,Central Toronto,2.0,Park,French Restaurant,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Cluster 4

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,West Toronto,3.0,Convenience Store,Park,Residential Building (Apartment / Condo),Sandwich Place,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Cluster 5

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4.0,Playground,Gym,Trail,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
